# Named Entity Recognition

In [1]:
from transformers import pipeline

In [2]:
nlp = pipeline("ner")


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing TFBertForTokenClassification: ['dropout_147']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english and are newly initialized: ['dropout_73']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge which is visible from the window."

In [4]:
print(nlp(sequence))

[{'word': 'Hu', 'score': 0.999578595161438, 'entity': 'I-ORG', 'index': 1}, {'word': '##gging', 'score': 0.9909763932228088, 'entity': 'I-ORG', 'index': 2}, {'word': 'Face', 'score': 0.9982224702835083, 'entity': 'I-ORG', 'index': 3}, {'word': 'Inc', 'score': 0.9994880557060242, 'entity': 'I-ORG', 'index': 4}, {'word': 'New', 'score': 0.9994344711303711, 'entity': 'I-LOC', 'index': 11}, {'word': 'York', 'score': 0.9993196129798889, 'entity': 'I-LOC', 'index': 12}, {'word': 'City', 'score': 0.9993793964385986, 'entity': 'I-LOC', 'index': 13}, {'word': 'D', 'score': 0.9862582087516785, 'entity': 'I-LOC', 'index': 19}, {'word': '##UM', 'score': 0.9514268636703491, 'entity': 'I-LOC', 'index': 20}, {'word': '##BO', 'score': 0.9336592555046082, 'entity': 'I-LOC', 'index': 21}, {'word': 'Manhattan', 'score': 0.9761654138565063, 'entity': 'I-LOC', 'index': 28}, {'word': 'Bridge', 'score': 0.9914628863334656, 'entity': 'I-LOC', 'index': 29}]


In [5]:
from transformers import TFAutoModelForTokenClassification, AutoTokenizer
import tensorflow as tf

model = TFAutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]

sequence = "Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very" \
           "close to the Manhattan Bridge."

# Bit of a hack to get the tokens with the special tokens
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="tf")

outputs = model(inputs)[0]
predictions = tf.argmax(outputs, axis=2)

All model checkpoint weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [6]:
print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())])

[('[CLS]', 'O'), ('Hu', 'I-ORG'), ('##gging', 'I-ORG'), ('Face', 'I-ORG'), ('Inc', 'I-ORG'), ('.', 'O'), ('is', 'O'), ('a', 'O'), ('company', 'O'), ('based', 'O'), ('in', 'O'), ('New', 'I-LOC'), ('York', 'I-LOC'), ('City', 'I-LOC'), ('.', 'O'), ('Its', 'O'), ('headquarters', 'O'), ('are', 'O'), ('in', 'O'), ('D', 'I-LOC'), ('##UM', 'I-LOC'), ('##BO', 'I-LOC'), (',', 'O'), ('therefore', 'O'), ('very', 'O'), ('##c', 'O'), ('##lose', 'O'), ('to', 'O'), ('the', 'O'), ('Manhattan', 'I-LOC'), ('Bridge', 'I-LOC'), ('.', 'O'), ('[SEP]', 'O')]
